# Evals API: 画像入力

このクックブックでは、画像ベースのタスクに対してOpenAIのEvalsフレームワークを使用する方法を説明します。Evals APIを活用して、**サンプリング**によってモデルが生成した応答を生成し、**モデル評価**（LLM as a Judge）を使用して、画像、プロンプト、参照回答に対するモデル応答を採点することで、画像とプロンプトに対するモデル生成応答を評価します。

この例では、モデルが以下のことをどの程度うまく実行できるかを評価します：
1. **画像に関するユーザープロンプトに対して適切な応答を生成する**
3. **高品質な応答を表す参照回答と一致する**

## 依存関係のインストール + セットアップ

In [1]:
# Install required packages
!pip install openai datasets pandas --quiet

In [ ]:
# Import libraries
from datasets import load_dataset
from openai import OpenAI
import os
import json
import time
import pandas as pd

## データセット準備

Hugging Faceでホストされている[VibeEval](https://huggingface.co/datasets/RekaAI/VibeEval)データセットを使用します。このデータセットには、ユーザープロンプト、付随する画像、および参照回答データのコレクションが含まれています。まず、データセットを読み込みます。

In [3]:
dataset = load_dataset("RekaAI/VibeEval")

関連するフィールドを抽出し、Evals APIでデータソースとして渡すためにjson形式で整理します。入力画像データは、WebのURLまたはbase64エンコードされた文字列の形式で提供できます。ここでは、提供されたWebのURLを使用します。

In [ ]:
evals_data_source = []

# select the first 3 examples in the dataset to use for this cookbook
for example in dataset["test"].select(range(3)):
    evals_data_source.append({
        "item": {
            "media_url": example["media_url"], # image web URL
            "reference": example["reference"], # reference answer
            "prompt": example["prompt"] # prompt
        }
    })

データソースリストを出力すると、各アイテムは以下のような形式になります：

```python
{
  "item": {
    "media_url": "https://storage.googleapis.com/reka-annotate.appspot.com/vibe-eval/difficulty-normal_food1_7e5c2cb9c8200d70.jpg"
    "reference": "This appears to be a classic Margherita pizza, which has the following ingredients..."
    "prompt": "What ingredients do I need to make this?"
  }
}
```

## Eval設定

データソースとタスクが準備できたので、evalを作成します。OpenAI Evals APIのドキュメントについては、[APIドキュメント](https://platform.openai.com/docs/evals/overview)をご覧ください。

In [ ]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

Evalsには「Eval」と「Run」の2つの部分があります。「Eval」では、データの期待される構造とテスト基準（grader）を定義します。

### データソース設定

私たちが収集したデータに基づいて、データソース設定は以下の通りです：

In [28]:
data_source_config = {
    "type": "custom",
    "item_schema": {
        "type": "object",
        "properties": {
          "media_url": { "type": "string" },
          "reference": { "type": "string" },
          "prompt": { "type": "string" }
        },
        "required": ["media_url", "reference", "prompt"]
      },
    "include_sample_schema": True, # enables sampling
}

### テスト基準

テスト基準として、グレーダー設定を行います。この例では、画像、参考回答、およびサンプルモデル応答（`sample`名前空間内）を受け取り、モデル応答が参考回答とどの程度一致しているか、および会話に対する一般的な適合性に基づいて0から1の間のスコアを出力するモデルグレーダーです。モデルグレーダーの詳細については、[API Grader docs](https://platform.openai.com/docs/api-reference/graders)をご覧ください。

効果的な評価を行うためには、データとグレーダーの両方を適切に設定することが重要です。そのため、グレーダーのプロンプトを反復的に改良していくことが必要になるでしょう。

**注意**: 画像URLフィールド / テンプレートは、画像として解釈されるために入力画像オブジェクトに配置する必要があります。そうでなければ、画像はテキスト文字列として解釈されます。

In [29]:
grader_config = {
	    "type": "score_model",
        "name": "Score Model Grader",
        "input":[
            {
                "role": "system",
		        "content": "You are an expert grader. Judge how well the model response suits the image and prompt as well as matches the meaniing of the reference answer. Output a score of 1 if great. If it's somewhat compatible, output a score around 0.5. Otherwise, give a score of 0."
	        },
	        {
		        "role": "user",
		        "content": [{ "type": "input_text", "text": "Prompt: {{ item.prompt }}."},
							{ "type": "input_image", "image_url": "{{ item.media_url }}", "detail": "auto" },
							{ "type": "input_text", "text": "Reference answer: {{ item.reference }}. Model response: {{ sample.output_text }}."}
				]
	        }
		],
		"pass_threshold": 0.9,
	    "range": [0, 1],
	    "model": "o4-mini" # model for grading; check that the model you use supports image inputs
	}

次に、evalオブジェクトを作成します。

In [ ]:
eval_object = client.evals.create(
        name="Image Grading",
        data_source_config=data_source_config,
        testing_criteria=[grader_config],
    )

## Eval Run

実行を作成するために、eval オブジェクト ID、データソース（つまり、先ほどコンパイルしたデータ）、およびモデルレスポンスを生成するためのサンプリングに使用するチャットメッセージ入力を渡します。なお、EvalsAPI は保存された補完や画像を含むレスポンスもデータソースとしてサポートしています。詳細については、[追加情報：ログデータソース](#additional-info-logs-data-source)セクションを参照してください。

この例で使用するサンプリングメッセージ入力は以下の通りです。

In [31]:
sampling_messages = [{
    "role": "user",
    "type": "message",
    "content": {
        "type": "input_text",
        "text": "{{ item.prompt }}"
      }
  },
  {
    "role": "user",
    "type": "message",
    "content": {
        "type": "input_image",
        "image_url": "{{ item.media_url }}",
        "detail": "auto"
    }
  }]

評価実行を開始します。

In [32]:
eval_run = client.evals.runs.create(
        name="Image Input Eval Run",
        eval_id=eval_object.id,
        data_source={
            "type": "responses", # sample using responses API
            "source": {
                "type": "file_content",
                "content": evals_data_source
            },
            "model": "gpt-4o-mini", # model used to generate the response; check that the model you use supports image inputs
            "input_messages": {
                "type": "template", 
                "template": sampling_messages}
        }
    )

## 投票結果の取得と表示

実行が完了したら、結果を確認できます。また、組織のOpenAI evalsダッシュボードで進行状況と結果を確認することもできます。

In [46]:
while True:
    run = client.evals.runs.retrieve(run_id=eval_run.id, eval_id=eval_object.id)
    if run.status == "completed" or run.status == "failed": # check if the run is finished
        output_items = list(client.evals.runs.output_items.list(
            run_id=run.id, eval_id=eval_object.id
        ))
        df = pd.DataFrame({
                "prompt": [item.datasource_item["prompt"]for item in output_items],
                "reference": [item.datasource_item["reference"] for item in output_items],
                "model_response": [item.sample.output[0].content for item in output_items],
                "grading_results": [item.results[0]["sample"]["output"][0]["content"]
                                    for item in output_items]
            })
        display(df)
        break
    time.sleep(5)

,prompt,reference,model_response,grading_results
0,Please provide latex code to replicate this table,Below is the latex code for your table:\n```te...,Certainly! Below is the LaTeX code to replicat...,"{""steps"":[{""description"":""Assess if the provid..."
1,What ingredients do I need to make this?,"This appears to be a classic Margherita pizza,...",To make a classic Margherita pizza like the on...,"{""steps"":[{""description"":""Check if model ident..."
2,Is this safe for a vegan to eat?,"Based on the image, this dish appears to be a ...",To determine if the dish is safe for a vegan t...,"{""steps"":[{""description"":""Compare model respon..."


### 個別の出力項目の表示

完全な出力項目を確認するには、以下のようにします。出力項目の構造は、APIドキュメントの[こちら](https://platform.openai.com/docs/api-reference/evals/run-output-item-object)で指定されています。

In [47]:
first_item = output_items[0]

print(json.dumps(dict(first_item), indent=2, default=str))

{
  "id": "outputitem_687833f102ec8191a6e53a5461b970c2",
  "created_at": 1752708081,
  "datasource_item": {
    "prompt": "Please provide latex code to replicate this table",
    "media_url": "https://storage.googleapis.com/reka-annotate.appspot.com/vibe-eval/difficulty-normal_table0_b312eea68bcd0de6.png",
    "reference": "Below is the latex code for your table:\n```tex\n\\begin{table}\n\\begin{tabular}{c c c c} \\hline  & \\(S2\\) & Expert & Layman & PoelM \\\\ \\cline{2-4} \\(S1\\) & Expert & \u2013 & 54.0 & 62.7 \\\\  & Layman & 46.0 & \u2013 & 60.7 \\\\  &,PoelM,LM,LM,LM,LM,LM,,L,M,,L,M,,L,M,,L,M,,,\u2013&39.3 \\\\\n[-1ex] \\end{tabular}\n\\end{table}\n```."
  },
  "datasource_item_id": 1,
  "eval_id": "eval_687833d68e888191bc4bd8b965368f22",
  "object": "eval.run.output_item",
  "results": [
    {
      "name": "Score Model Grader-73fe48a0-8090-46eb-aa8e-d426ad074eb3",
      "sample": {
        "input": [
          {
            "role": "system",
            "content": "You are a

## 追加情報: ログデータソース

前述したように、EvalsAPIは画像を含むログ（つまり、保存された補完や応答）をデータソースとしてサポートしています。この機能を使用するには、評価設定を以下のように変更してください：

評価の作成
  - `data_source_config = { "type": "logs" }` を設定する
  - `grader_config` のテンプレート化を修正して、ログの入力と出力を示す `{{item.input}}` および/または `{{sample.output_text}}` を使用する

評価実行の作成
  - 評価実行に対応するログを取得するために使用される `data_source` フィールドにフィルターを指定する（詳細については[ドキュメント](https://platform.openai.com/docs/api-reference/evals/createRun)を参照）

## まとめ

このクックブックでは、OpenAI Evals APIを使用して画像ベースのタスクを評価するワークフローについて説明しました。サンプリングとモデル採点の両方で画像入力機能を使用することで、このタスクの評価プロセスを効率化することができました。

OCR精度、画像生成の採点など、皆さん独自の画像ベースのユースケースにこの手法を拡張していただけることを楽しみにしています！